# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Preparation

In [3]:
%run "Utilities/Preparation.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (24,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Functions
<a id=functions />

In [4]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    columnAnswers = answers.T
    columnAnswers.columns = ['answers']
    return columnAnswers

def getCorrections( userId):
    columnAnswers = getAnswers( userId )
    
    columnAnswers['correct'] = columnAnswers['answers']

    for index in range(0, 10):
        columnAnswers['correct'][index] = 'NaN'
    for index in range(10, len(columnAnswers.values)-1):
        columnAnswers['correct'][index] = str(columnAnswers['answers'][index]).startswith(str(correctAnswers['correctAnswer'][index]))
    columnAnswers['correct'][len(columnAnswers.values)-1] = 'NaN'
    return columnAnswers

#function that returns the score from user id
def getScore( userId ):
    if hasAnswered( userId ):
        columnAnswers = getCorrections( userId)
        score = (columnAnswers['correct']).value_counts()[True]
        return score
    else:
        return np.NaN

#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    columnAnswers = getCorrections( userId)
    questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))
    
    for index in range(0, len(questionnaireValidatedCheckpointsPerQuestion)):
        if columnAnswers['correct'][index]==True:
            questionnaireValidatedCheckpointsPerQuestion[index] = checkpointQuestionMatching['checkpoint'][index]
        else:
            questionnaireValidatedCheckpointsPerQuestion[index] = ''
    
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpointsPerQuestion.unique()
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints[questionnaireValidatedCheckpoints!='']
    questionnaireValidatedCheckpoints = pd.Series(questionnaireValidatedCheckpoints)
    questionnaireValidatedCheckpoints = questionnaireValidatedCheckpoints.sort_values()
    questionnaireValidatedCheckpoints.index = range(0, len(questionnaireValidatedCheckpoints))
    return questionnaireValidatedCheckpoints

def getNonValidated( checkpoints ):
    return pd.Series(np.setdiff1d(checkpointQuestionMatching['checkpoint'].values, checkpoints.values))

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)

